**取得google drive認證**

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**設定path(本目錄的絕對路徑)**

**下載 SimpleITK套件**

In [0]:
path='/content/drive/My Drive/CT'
!pip install SimpleITK

import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.ndimage import zoom
import cv2

     |████████████████████████████████| 42.5MB 98kB/s 


In [0]:
#以迴圈逐筆讀進資料
#取得每筆資料的space後使用zoom更改像素pixel size=(1,1,1)
#使用resize 更改x,y大小
for i in range(1,16):
  print('第',i,'筆')
  if i<10:
    image = sitk.ReadImage(path+"/xVertSeg.v1/Data1/images/image00"+str(i)+".mhd")
    mask = sitk.ReadImage(path+"/xVertSeg.v1/Data1/masks/mask00"+str(i)+".mhd")
  else:
    image = sitk.ReadImage(path+"/xVertSeg.v1/Data1/images/image0"+str(i)+".mhd")
    mask = sitk.ReadImage(path+"/xVertSeg.v1/Data1/masks/mask0"+str(i)+".mhd")

  images=sitk.GetArrayFromImage(image)
  masks=sitk.GetArrayFromImage(mask)

  print("images shape:",images.shape)
  print("masks shape:",masks.shape)      

  origin_image = image.GetOrigin()
  spacing_image = image.GetSpacing()

  origin_mask = mask.GetOrigin()
  spacing_mask = mask.GetSpacing()

  print("spacing_mask:",spacing_mask) #x,y,z  

  new_images = zoom(images, (spacing_image[2],spacing_image[0], spacing_image[1]))      
  new_masks = zoom(masks, (spacing_mask[2],spacing_image[0], spacing_image[1]),order=0)

  print("new mask unique:",np.unique(new_masks))
  print("new images shape:",new_images.shape)
  print("new masks shape:",new_masks.shape)

  new_images=np.transpose(new_images, (1, 2, 0))         #(z,x,y) to (x,y,z)
  new_masks=np.transpose(new_masks, (1, 2, 0))         #(z,x,y) to (x,y,z)
  images_shape=new_images.shape
  masks_shape=new_masks.shape

  images_stack_sm = np.zeros((128,128,images_shape[2]),dtype=np.float64)
  mask_stack_sm = np.zeros((128,128,masks_shape[2]),dtype=np.float64)

  for idx in range(images_shape[2]):
    img = new_images[:, :,idx]
    img_sm = cv2.resize(img, (128, 128), interpolation=cv2.INTER_NEAREST)                 
    images_stack_sm[:, :,idx] = img_sm

  for idx in range(masks_shape[2]):
    img = new_masks[:, :,idx]
    img_sm = cv2.resize(img, (128, 128), interpolation=cv2.INTER_NEAREST)                 
    mask_stack_sm[:, :,idx] = img_sm

  print("resize_images_shape:",images_stack_sm.shape)
  print("resize_masks_shape:",mask_stack_sm.shape)

  #判斷z軸是否大於256
  if images_stack_sm.shape[2]>256:
    sitk_images=sitk.GetImageFromArray(images_stack_sm[:,:,-256:])
    sitk_masks=sitk.GetImageFromArray(mask_stack_sm[:,:,-256:])

  else:
    new_img_data_train = np.zeros((128, 128, 256),dtype=np.float32)
    new_img_data_masks = np.zeros((128, 128, 256),dtype=np.float32)
    new_img_data_train[:,:,:images_stack_sm.shape[2]]=images_stack_sm
    new_img_data_masks[:,:,:mask_stack_sm.shape[2]]=mask_stack_sm
    sitk_images=sitk.GetImageFromArray(new_img_data_train)
    sitk_masks=sitk.GetImageFromArray(new_img_data_masks)

  image_filename=path+"/xVertSeg.v1/Data1/resize_images_128/reize_image"+str(i)+".mhd"
  mask_filename=path+"/xVertSeg.v1/Data1/resize_masks_128/reize_mask"+str(i)+".mhd"
  sitk.WriteImage(sitk_images,image_filename)
  sitk.WriteImage(sitk_masks,mask_filename)

第 1 筆
images shape: (200, 1024, 1024)
masks shape: (200, 1024, 1024)
spacing_mask: (0.41362, 0.41362, 1.4506)
new mask unique: [  0 200 210 220 230 240]
new images shape: (290, 424, 424)
new masks shape: (290, 424, 424)
resize_images_shape: (128, 128, 290)
resize_masks_shape: (128, 128, 290)
第 2 筆
images shape: (250, 1024, 1024)
masks shape: (250, 1024, 1024)
spacing_mask: (0.43207, 0.43207, 1.2895)
new mask unique: [  0 200 210 220 230 240]
new images shape: (322, 442, 442)
new masks shape: (322, 442, 442)
resize_images_shape: (128, 128, 322)
resize_masks_shape: (128, 128, 322)
第 3 筆
images shape: (340, 1024, 1024)
masks shape: (340, 1024, 1024)
spacing_mask: (0.5407, 0.5407, 1.1924)
new mask unique: [  0 200 210 220 230 240]
new images shape: (405, 554, 554)
new masks shape: (405, 554, 554)
resize_images_shape: (128, 128, 405)
resize_masks_shape: (128, 128, 405)
第 4 筆
images shape: (170, 1024, 1024)
masks shape: (170, 1024, 1024)
spacing_mask: (0.42524, 0.42524, 1.2837)
new mask uniq